# Operating DNN Pixel Classifier Model


This module is about training/testing a deep neural network, using a basic architecture of a pixel classifier ( originally proposed by <sup>   [[ciresan2012]](http://people.idsia.ch/~ciresan/data/nips2012.pdf)</sup>).

+ The specific neural network model used, is defined elsewhere (`net/utils/models.py`).



![DNN Model](https://i.imgur.com/hC3AL1b.png)
This image gives an overview of the basic architecture.

#### Basically the NN is constructed from densifying pyramidal blocks of (conv-relu; maxpool) NN layers.

#### The first layer can have **copied weights**, taken from a previously trained AEN model.


Using this notebook, a DNN model can be:
 1. Trained from a lmdb training database, with specific train parameters;
 2. Tested using a full **interim** image set input, to generate _"cleared"_ **DNN Corrected** representations.





------------------

# Importing modules

Importing all dependent python functions for training a DNN.

The main computational frameworks used were:
+ [Keras](https://keras.io/) as a high level DNN Python API.
+ [Tensorflow](https://www.tensorflow.org/) as a low level GPU processing framework.

Additionally [LMDB](https://lmdb.readthedocs.io/) was used as a database API for data managment.




In [1]:
import os
import lmdb
import sys
import time
from functools import partial
import numpy as np
from numpy import array as arr

import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard,EarlyStopping,LambdaCallback
from keras.models import load_model

from keras.optimizers import SGD
import keras.backend.tensorflow_backend as KTF


sys.path.append("../..")

import dnn_segmentation.net.utils as net_utils


from dnn_segmentation.net.utils.train_h import iterate_indef
 


from dnn_segmentation.net.utils.train_h import \
    quick_test,get_session,draw_onfirst_epoch,save_relevant,\
    use_best_model,use_num_model

from dnn_segmentation.net.utils.models import \
    get_2k_image_good_convmodel,get_2k_twopath_simple,\
    get_2k_twopath_twoconv,get_2k_image_pretrained, \
    get_2k_image_2layer_convnetmodel
    
print('Every import is succesful !')

Using TensorFlow backend.


Every import is succesful !


## Setting training/testing parameters

Here all training/testing parameters before processing are set.

Mainly they can be divided in:
1. Boolean Configuration Flag  

   + Train a **New** DNN Model, 
    
   + **Predict** with an existing model to _"correct"_ a full image set.
    
1. Model, Data **path locations**;
2. Input image/patch info;
3. DNN training info
    1. optimiser
    2. batch size
    3. num of train epochs
4. Other programatical processing flags


In [2]:
# DNN Main Processing FLAG
newTrain=False



# ------------------------------------------------------------------
# ------------------------------------------------------------------
# ------------------------------------------------------------------
# ------------------------------------------------------------------



np.random.seed(None)
KTF.set_session(get_session(0.8))


model_locs=r'../test_data'
log_loc=r'../test_data/logs'  # Not used, tensorflow log location


val_lmdb=r'######/val_db'
train_lmdb=r'####/train_db'

img_w,img_h=45,45
big_img_size=2048

# optimizer = SGD(lr=0.0001, decay=0.0005, 
# momentum=0.9, nesterov=True)
optimizer='adadelta'

loss_func='categorical_crossentropy' 

batch_size=500
epoch_count=3


#  Random short nametag, for every new model experiment.
quick_str=''.join(map(chr,np.random.randint(97,97+26,(4,))) )





split_patches=True  # true for autoenc

single_map_autoenc=False #true for single autoenc
numchannel_autoenc=3 # true for..1


epoch_toDraw=None # None for best epoch
start_patch48=True  # true for autoenc



## Aditional setup parameters

* Setting **pretraining** parameters
 1. Crbm based pretraining
 2. Autoenc based pretraining
* Setting validation frequency
* Other flags
 

In [3]:



added_stuff={
            # 'path_pretrain':
            #      r'crmb_model_valloss_3333.npy',
    'use_crbm':False,
    'use_autoenc':True,
             'autoenc_loc':r'../test_data/weights_autoenc_gced.00-0.1958.hdf5',
             'autoenc_layer':'convolution2d_1'}

train_autoenc_loc=r'../test_data/autoenc_train'
reduce_valSize_Fac=0.2 # 1

do_quick_after_train=False
do_test_after_one=False
test_after_train_fullset=False


do_whole_pic=True
draw_img_ext='*.tif'
val_freq=122300


weights_prepend='weights_data_'+quick_str


print('New Model is: ',weights_prepend)

New Model is:  weights_data_vjat


## DNN Correcting Images


### (skip if new DNN training) 

This codeblock is for _"testing"_ a previously trained DNN module.


For start, some configurtation parameters:
+ Location path of data, image sets;
+ Correcting DNN model location;
+ Correcting experiment tag;
+ Image set iterator

With the configuration, 

the code iteratively uses the DNN to _"clean"_ the input **interim** patches, (subset from the full EM **preprocessed** images)


In [4]:
test_parDikt={
    'patch_size':45,
    'img_path':r'../test_data/possible_dnn_input',
    'groundtruth':r'../test_data/Consensus corrected',
    'interim':r'../test_data/possible_dnn_input'}


save_model_loc = r'../test_data/weights_wdog.hdf5'

full_set_description='MLset01_images'
set_iter=zip(range(0,62,1),range(1,62,1),range(1))

if not newTrain:
    print('Continued Training')

    # model=load_model(save_model_loc,
        #custom_objects={'CRBMPretrained':CRBMPretrained})
    model=load_model(save_model_loc,
                     custom_objects={'AutoEncoderLayer':
                            net_utils.AutoEncoderLayer.AutoEncoderLayer})

    quick_oldstr=save_model_loc[:save_model_loc.index('.')]
    quick_oldstr+=full_set_description

    for older,newer,xx in set_iter:

        test_parDikt.update({'from': older, 'to': newer})

        quick_test(model, quick_oldstr, big_img_size, do_all=do_whole_pic,
                   draw_img_ext=draw_img_ext, test_batch_size=50,
                   split_map=split_patches,
                   conf_dikt=test_parDikt,two_patch_instance=start_patch48,
                  prediction_basedir='../data')
    sys.exit('I\'m done continued testing')


Continued Training
['../test_data/possible_dnn_input/sp13726-img05-interim.tif'] 0 1 ../test_data/possible_dnn_input
Drawing on these:  1 ['../test_data/possible_dnn_input/sp13726-img05-interim.tif']
Testing started - 3.498679
Running on img:sp13726-img05-interim.tif
Starting patching on image:  sp13726-img05  ...



KeyboardInterrupt: 

## New Training

### setup dnn specific parameters

1. Getting the model architecture (specified elsewhere);
2. Setting training callbacks, currently active:
    1. Save model every training epoch
    3. Early stop training, if val-acc smoothens.
    5. Draw full EM image after 1 epoch trained
    6. Log Losses during training
 

In [10]:

if newTrain:
    print('New Training')
    model = get_2k_image_pretrained(img_w, img_h,added_stuff,\
                                    ch_add=numchannel_autoenc)

    model_epoch=0
    model.compile(loss=loss_func,
                  optimizer=optimizer,
                  metrics=['accuracy'])

## Implemented in .py files
config_text='#####NOT ACTIVE####'

#save_relevant('saved_quickmodels', quick_str,\
#                          just_return=True)



# reduce_lr_call=ReduceLROnPlateau(monitor='val_acc',factor=0.2,
#                                  patience=3,cooldown=2,verbose=1)
save_model_call=ModelCheckpoint(os.path.join(
        model_locs,weights_prepend+'.{epoch:02d}-{val_acc:.4f}.hdf5'),
                                verbose=1,monitor='val_acc'
                                )

earlystop_call=EarlyStopping(monitor='val_acc', 
                             min_delta=0.0001, patience=5,
                             verbose=1, mode='auto')
# tensor_call=TensorBoard(log_dir=log_loc, histogram_freq=3,
# write_graph=True, write_images=True)





dodraw_afterone=LambdaCallback(
    on_epoch_end=partial(draw_onfirst_epoch,
                         model=model,
                         big_img_size=big_img_size,
                         do_test=do_test_after_one,
                         quick_str=quick_str,
                         str_to_save=config_text,
                         split_map=split_patches))



# log_losses=net_utils.LogLossesCallback.LogLossesCallback(val_freq//batch_size,(val_freq*3)//batch_size,model_id=quick_str,
#                              save_loc=train_autoenc_loc,save_model=r'D:\data_projects\neuron_fibers_data\autoenc')

all_calls=[save_model_call,earlystop_call,dodraw_afterone]







## Executing Training


* Actually training the DNN, using lmdb databases to obtain **input data**.

Code stops after the specified training epochs...

In [ ]:



start_t=time.time()


lmdbval_env=lmdb.open(val_lmdb)
lmdbval_txn=lmdbval_env.begin()

lmdbtrain_env=lmdb.open(train_lmdb)
lmdbtrain_txn=lmdbtrain_env.begin()
train_size=lmdbtrain_env.stat()['entries']
val_size=int(lmdbval_env.stat()['entries']*reduce_valSize_Fac)


oneI=iterate_indef(lmdbval_txn, batch_size, img_w *2 if start_patch48 else img_w, img_h, two_patch_instance=start_patch48,
              do_continuous=True,
                   do_single_patch=single_map_autoenc,
                   split_map=split_patches,return_dnn_annotation=True)

raw_patches=[next(oneI) for ind in range(4)]
val_data_x,val_data_y=arr([i[0] for i in raw_patches]),arr([i[1] for i in raw_patches])
oneI=None
log_losses.val_data=val_data_x.reshape((4*batch_size,)+val_data_x.shape[2:]),val_data_y.reshape((4*batch_size,)+val_data_y.shape[2:])

model.fit_generator(
    iterate_indef(lmdbtrain_txn,batch_size,img_w *2 if start_patch48 else img_w,img_h,
                  do_continuous=True,
        do_single_patch=single_map_autoenc,
                  split_map=split_patches,two_patch_instance=start_patch48,return_dnn_annotation=True),
            samples_per_epoch=train_size-train_size%batch_size,
                    nb_epoch=epoch_count,

          verbose=1,
        callbacks=all_calls,
        validation_data= \
        iterate_indef(lmdbval_txn, batch_size, img_w *2 if start_patch48 else img_w, img_h,
                    do_single_patch=single_map_autoenc,
                      do_continuous=True,split_map=split_patches,two_patch_instance=start_patch48,return_dnn_annotation=True),
        nb_val_samples=val_size-val_size%batch_size )


lmdbtrain_env.close()

# score = model.evaluate_generator(
#     iterate_indef(lmdbval_txn, batch_size, img_w, img_h, do_continuous=True),
#         val_samples=val_size-val_size%batch_size, verbose=1)
#
# print('Test score:', score[0])
# print('Test accuracy:', score[1])


lmdbval_env.close()

print('Fully done with training !')


## Possibly testing DNN after training

* Code to draw images after the training, specifically using a **CRBM** pretrained layer

In [0]:


if do_quick_after_train:
    best_model_loc=use_num_model(quick_str,num_model=epoch_toDraw)
    best_model=load_model(best_model_loc,custom_objects={'CRBMPretrained':CRBMPretrained})

    quick_test(best_model,quick_str,big_img_size,do_all=do_whole_pic
               ,split_map=split_patches,draw_img_ext=draw_img_ext)
    # save_relevant('saved_quickmodels',quick_str)

if test_after_train_fullset:

    quick_str += 'fullset'
    for older, newer in zip(range(0, 31, 8), range(8, 33, 8)):
        test_parDikt.update({'from':older,'to':newer})

        quick_test(model, quick_str, big_img_size, do_all=do_whole_pic, 
                   draw_img_ext=draw_img_ext, test_batch_size=500,
                   split_map=split_patches,
                   conf_dikt=test_parDikt)

print('time duration is: ',time.time()-start_t)

